In [15]:
!pip install spotipy
!pip install python-dotenv

from pathlib import Path
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path('../.env'))

True

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# spotify credentials for the application:
# "Related Artist Network Visualizer"
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = 'f268f13219af4bf5b448ee8d07121e14'
# create a credential manager and api layer
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
# the spotify id for the artist Bassnectar
bassnectar = '1JPy5PsJtkhftfdr6saN2i'

In [11]:
name = sp.artist(bassnectar)

In [12]:
name

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1JPy5PsJtkhftfdr6saN2i'},
 'followers': {'href': None, 'total': 583275},
 'genres': ['breakbeat',
  'brostep',
  'edm',
  'electro house',
  'electronic trap',
  'glitch hop'],
 'href': 'https://api.spotify.com/v1/artists/1JPy5PsJtkhftfdr6saN2i',
 'id': '1JPy5PsJtkhftfdr6saN2i',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/5c3031087d35a465f8046cb7e94e80a30b23cb19',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/3f96dfc0f40f4142307445186db5669070d94a46',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/8ef055e1595a073806b1a91e96a6c1d73cf835a6',
   'width': 160}],
 'name': 'Bassnectar',
 'popularity': 63,
 'type': 'artist',
 'uri': 'spotify:artist:1JPy5PsJtkhftfdr6saN2i'}